In [2]:
import json
import numpy as np
import pickle
import re
import string
import nltk
import pandas as pd
import spacy
import re
import string
from gensim import matutils, models
import scipy.sparse
from collections import Counter
from sklearn.feature_extraction import text 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
spacy.prefer_gpu()
import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)


In [3]:
from glob import glob
import re
import string
import funcy as fp
from gensim import models
from gensim.corpora import Dictionary, MmCorpus
import nltk
import pandas as pd

In [4]:
data_df = pd.read_pickle('lemmatized_bigstrings.pickle')
data_df['index'] = data_df.index
data_df

,text,index
wsb,future apple share gran kid ask not buy appl...,wsb
science,child motivate achieve maximum reward extent...,science
ama,tldr fuck want tell people internet lie funn...,ama
askreddit,victorias secret originally suppose place man ...,askreddit
worldnews,post leave purge ok context facebook estimat...,worldnews
funny,want robot ferret haha suck fool get robotic f...,funny
dankmemes,meme month seed nomination july nomination...,dankmemes
memes,redditmc open staff position builder helper mo...,memes
nosleep,look like story click reminder check later g...,nosleep
psychology,decision mental health care canada study ...,psychology


In [9]:
data_df = pd.read_pickle('cleaned_list_df.pickle')
data_df

,0,1,2,3,4,5,6,7,8,9,...,101131,101132,101133,101134,101135,101136,101137,101138,101139,101140
wsb,in the future when apple is at a share your g...,you ever stare at your account and think wow h...,buy at the top gang 🤡,almost responded to an email with thanks retar...,top trending tickers sentiments and options on...,some year old corn futures trader whos been a...,blew my account up today feels bad lost over ...,overnight gap up to ath secured,usually my balls tingle the day before a signi...,wsb after dark is going to be wild tonight i h...,...,none,none,none,none,none,none,none,none,none,none
science,the children were not motivated by achieving ...,also proved children like to gamble according ...,kind of like how gamers go through rpg games e...,i teach elementary pe to year olds one of th...,wouldnt this make lootmystery boxes even worse...,i find that very encouraging,you could have a boat or whats in this box,like when playing diablo scanned every dungeon...,i have a and year old boy by not shoving scr...,basically how ppl play rpgs they explore every...,...,none,none,none,none,none,none,none,none,none,none
ama,tldr do whatever the fuck you want,so youre telling me people on the internet lie,,this is so funny this subreddit got fucked by ...,a note about celebrity request bans when riama...,do you like cantaloupe,what does ama stand for,i liked that guy he seemed like a really nice...,hmmm ok thnx for clarifying all that but one q...,so basically rama is a shitty knockoff of rima...,...,none,none,none,none,none,none,none,none,none,none
askreddit,victorias secret was originally supposed to be...,the ford mustang it was supposed to be a car ...,heels if i remember correctly,is this how family feud gets new questions and...,skirts used to be worn in battle by men for mo...,ugg boots were originally worn by male surfers...,filtered cigarettes in general but marlboro ma...,those leg stirrups they use at the gynocologis...,my hoodie apparently,pink and blue were originally considered color...,...,shut up and take all my caps,is that butcher pete i see,happy cake day and yes maybe at bars close to ...,heppy cake day,not his ghost tho,maybe,is it my cake day i have always wondered how p...,theres a cake emoji next to your username on y...,i think my problem is that im using narwhal ed...,lol
worldnews,so was there any posts left after the purge,ok just for some context facebook has an esti...,they still havent taken down my post about mar...,why does facebook have a fake news problem be...,a lot of karens are going to be be saying they...,i bought a mask that arrived in a box that had...,this is the best tldr i could make reduced by...,why is my aunt and my crazy high school friend...,,seems low,...,none,none,none,none,none,none,none,none,none,none
funny,but i wanted the robot ferret,haha suck it you fools i already got my roboti...,this not funny rule number oh i forget carry on,but the nigerian guy is legit right,bruh when i was younger i went to a farm this ...,how do i sign up for the ferret,i was looking forward to the robotic ferret m...,mine came from a nigerian prince so its totall...,i clicked on one of those links but i did not ...,ive gotten a few and always assumed they were ...,...,none,none,none,none,none,none,none,none,none,none
dankmemes,meme of the month seeding and nominations for ...,dont have any examples and im not sure if it c...,certified hood classic,arnold schwarzerunner godtier,el dorado explanation,alignment charts sorta popular and cool to see...,the guys on a couch explaining something to t...,,where banana,chad meme,...,none,none,none,none,none,none,none,none,none,none
memes,redditmc is opening staff positions builders h...,using this to see if i have new flair,cries in bedrock edition,thank you thats very cash money of you,it wont let me connect to the server im using ...,how many have i infected,,why the fuck do i have negative karma i only p...,i need to get comment karma to be able to post...,

In [5]:
cvn = TfidfVectorizer(ngram_range=(1,2))
data_cvn = cvn.fit_transform(data_df.text[:1])
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names()).T
# data_dtmn.columns = data_df.index
data_dtmn

,0
aaa,0.000414
aaa informal,0.000083
aaa literally,0.000083
aaa loan,0.000083
aaa mean,0.000083
...,...
𝓭𝓸𝔀𝓷 be,0.000083
𝕵𝖊𝖘𝖚𝖘,0.000083
𝕵𝖊𝖘𝖚𝖘 𝕷𝖊𝖆𝖉𝖊𝖗𝖇𝖔𝖆𝖗𝖉,0.000083
𝕷𝖊𝖆𝖉𝖊𝖗𝖇𝖔𝖆𝖗𝖉,0.000083


In [6]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())
word2id = dict((k, v) for k, v in cvn.vocabulary_.items())

In [7]:
# Let's start with 2 topics
ldan = models.LdaMulticore(corpus=corpusn, num_topics=50, id2word=id2wordn, passes=50)
ldan.print_topics()

INFO - 12:57:22: using symmetric alpha at 0.02
INFO - 12:57:22: using symmetric eta at 0.02
INFO - 12:57:22: using serial LDA version on this node
INFO - 12:57:22: running online LDA training, 50 topics, 50 passes over the supplied corpus of 1 documents, updating every 30000 documents, evaluating every ~1 documents, iterating 50x with a convergence threshold of 0.001000
INFO - 12:57:22: training LDA model using 15 processes
INFO - 12:57:22: PROGRESS: pass 0, dispatched chunk #0 = documents up to #1/1, outstanding queue size 1
INFO - 12:57:24: topic #36 (0.020): 0.000*"probably chance" + 0.000*"probably close" + 0.000*"probably busy" + 0.000*"probably buy" + 0.000*"probably call" + 0.000*"probably care" + 0.000*"probably careful" + 0.000*"probably catch" + 0.000*"probably cause" + 0.000*"probably bump"
INFO - 12:57:24: topic #37 (0.020): 0.000*"probably chance" + 0.000*"probably close" + 0.000*"probably busy" + 0.000*"probably buy" + 0.000*"probably call" + 0.000*"probably care" + 0.000

INFO - 12:57:37: topic #24 (0.020): 0.000*"probably chance" + 0.000*"probably close" + 0.000*"probably busy" + 0.000*"probably buy" + 0.000*"probably call" + 0.000*"probably care" + 0.000*"probably careful" + 0.000*"probably catch" + 0.000*"probably cause" + 0.000*"probably bump"
INFO - 12:57:37: topic diff=0.000275, rho=0.447191
INFO - 12:57:37: -57.076 per-word bound, 151953115937437984.0 perplexity estimate based on a held-out corpus of 1 documents with 39 words
INFO - 12:57:37: PROGRESS: pass 5, dispatched chunk #0 = documents up to #1/1, outstanding queue size 1
INFO - 12:57:39: topic #6 (0.020): 0.000*"probably chance" + 0.000*"probably close" + 0.000*"probably busy" + 0.000*"probably buy" + 0.000*"probably call" + 0.000*"probably care" + 0.000*"probably careful" + 0.000*"probably catch" + 0.000*"probably cause" + 0.000*"probably bump"
INFO - 12:57:39: topic #11 (0.020): 0.000*"probably chance" + 0.000*"probably close" + 0.000*"probably busy" + 0.000*"probably buy" + 0.000*"proba

INFO - 12:57:51: topic #38 (0.020): 0.000*"probably chance" + 0.000*"probably close" + 0.000*"probably busy" + 0.000*"probably buy" + 0.000*"probably call" + 0.000*"probably care" + 0.000*"probably careful" + 0.000*"probably catch" + 0.000*"probably cause" + 0.000*"probably bump"
INFO - 12:57:51: topic diff=0.000019, rho=0.316220
INFO - 12:57:52: -58.206 per-word bound, 332374093936046720.0 perplexity estimate based on a held-out corpus of 1 documents with 39 words
INFO - 12:57:52: PROGRESS: pass 10, dispatched chunk #0 = documents up to #1/1, outstanding queue size 1
INFO - 12:57:54: topic #43 (0.020): 0.000*"probably chance" + 0.000*"probably close" + 0.000*"probably busy" + 0.000*"probably buy" + 0.000*"probably call" + 0.000*"probably care" + 0.000*"probably careful" + 0.000*"probably catch" + 0.000*"probably cause" + 0.000*"probably bump"
INFO - 12:57:54: topic #46 (0.020): 0.000*"probably chance" + 0.000*"probably close" + 0.000*"probably busy" + 0.000*"probably buy" + 0.000*"pro

INFO - 12:58:06: topic #34 (0.020): 0.000*"probably chance" + 0.000*"probably close" + 0.000*"probably busy" + 0.000*"probably buy" + 0.000*"probably call" + 0.000*"probably care" + 0.000*"probably careful" + 0.000*"probably catch" + 0.000*"probably cause" + 0.000*"probably bump"
INFO - 12:58:06: topic #43 (0.020): 0.000*"probably chance" + 0.000*"probably close" + 0.000*"probably busy" + 0.000*"probably buy" + 0.000*"probably call" + 0.000*"probably care" + 0.000*"probably careful" + 0.000*"probably catch" + 0.000*"probably cause" + 0.000*"probably bump"
INFO - 12:58:06: topic diff=0.000003, rho=0.258195
INFO - 12:58:07: -58.467 per-word bound, 398449834892300352.0 perplexity estimate based on a held-out corpus of 1 documents with 39 words
INFO - 12:58:07: PROGRESS: pass 15, dispatched chunk #0 = documents up to #1/1, outstanding queue size 1
INFO - 12:58:09: topic #4 (0.020): 0.000*"probably chance" + 0.000*"probably close" + 0.000*"probably busy" + 0.000*"probably buy" + 0.000*"prob

INFO - 12:58:21: topic #28 (0.020): 0.000*"probably chance" + 0.000*"probably close" + 0.000*"probably busy" + 0.000*"probably buy" + 0.000*"probably call" + 0.000*"probably care" + 0.000*"probably careful" + 0.000*"probably catch" + 0.000*"probably cause" + 0.000*"probably bump"
INFO - 12:58:21: topic #46 (0.020): 0.000*"probably chance" + 0.000*"probably close" + 0.000*"probably busy" + 0.000*"probably buy" + 0.000*"probably call" + 0.000*"probably care" + 0.000*"probably careful" + 0.000*"probably catch" + 0.000*"probably cause" + 0.000*"probably bump"
INFO - 12:58:21: topic #8 (0.020): 0.000*"probably chance" + 0.000*"probably close" + 0.000*"probably busy" + 0.000*"probably buy" + 0.000*"probably call" + 0.000*"probably care" + 0.000*"probably careful" + 0.000*"probably catch" + 0.000*"probably cause" + 0.000*"probably bump"
INFO - 12:58:21: topic diff=0.000001, rho=0.223604
INFO - 12:58:22: -58.550 per-word bound, 422070390798289024.0 perplexity estimate based on a held-out corpu

...

INFO - 12:59:05: topic #19 (0.020): 0.000*"probably chance" + 0.000*"probably close" + 0.000*"probably busy" + 0.000*"probably buy" + 0.000*"probably call" + 0.000*"probably care" + 0.000*"probably careful" + 0.000*"probably catch" + 0.000*"probably cause" + 0.000*"probably bump"
INFO - 12:59:05: topic #22 (0.020): 0.000*"probably chance" + 0.000*"probably close" + 0.000*"probably busy" + 0.000*"probably buy" + 0.000*"probably call" + 0.000*"probably care" + 0.000*"probably careful" + 0.000*"probably catch" + 0.000*"probably cause" + 0.000*"probably bump"
INFO - 12:59:05: topic #18 (0.020): 0.000*"not" + 0.000*"buy" + 0.000*"go" + 0.000*"sell" + 0.000*"call" + 0.000*"be" + 0.000*"like" + 0.000*"get" + 0.000*"think" + 0.000*"fuck"
INFO - 12:59:05: topic #13 (0.020): 0.000*"probably chance" + 0.000*"probably close" + 0.000*"probably busy" + 0.000*"probably buy" + 0.000*"probably call" + 0.000*"probably care" + 0.000*"probably careful" + 0.000*"probably catch" + 0.000*"probably cause" + 0

INFO - 12:59:20: topic #42 (0.020): 0.000*"probably chance" + 0.000*"probably close" + 0.000*"probably busy" + 0.000*"probably buy" + 0.000*"probably call" + 0.000*"probably care" + 0.000*"probably careful" + 0.000*"probably catch" + 0.000*"probably cause" + 0.000*"probably bump"
INFO - 12:59:20: topic #16 (0.020): 0.000*"probably chance" + 0.000*"probably close" + 0.000*"probably busy" + 0.000*"probably buy" + 0.000*"probably call" + 0.000*"probably care" + 0.000*"probably careful" + 0.000*"probably catch" + 0.000*"probably cause" + 0.000*"probably bump"
INFO - 12:59:20: topic #30 (0.020): 0.000*"probably chance" + 0.000*"probably close" + 0.000*"probably busy" + 0.000*"probably buy" + 0.000*"probably call" + 0.000*"probably care" + 0.000*"probably careful" + 0.000*"probably catch" + 0.000*"probably cause" + 0.000*"probably bump"
INFO - 12:59:20: topic diff=0.000000, rho=0.158113
INFO - 12:59:21: -58.842 per-word bound, 516521646739938688.0 perplexity estimate based on a held-out corp

INFO - 12:59:34: topic #33 (0.020): 0.000*"probably chance" + 0.000*"probably close" + 0.000*"probably busy" + 0.000*"probably buy" + 0.000*"probably call" + 0.000*"probably care" + 0.000*"probably careful" + 0.000*"probably catch" + 0.000*"probably cause" + 0.000*"probably bump"
INFO - 12:59:34: topic #14 (0.020): 0.000*"probably chance" + 0.000*"probably close" + 0.000*"probably busy" + 0.000*"probably buy" + 0.000*"probably call" + 0.000*"probably care" + 0.000*"probably careful" + 0.000*"probably catch" + 0.000*"probably cause" + 0.000*"probably bump"
INFO - 12:59:34: topic #17 (0.020): 0.000*"probably chance" + 0.000*"probably close" + 0.000*"probably busy" + 0.000*"probably buy" + 0.000*"probably call" + 0.000*"probably care" + 0.000*"probably careful" + 0.000*"probably catch" + 0.000*"probably cause" + 0.000*"probably bump"
INFO - 12:59:34: topic diff=0.000000, rho=0.149070
INFO - 12:59:35: -58.884 per-word bound, 531758081706786112.0 perplexity estimate based on a held-out corp

INFO - 12:59:49: topic #49 (0.020): 0.000*"probably chance" + 0.000*"probably close" + 0.000*"probably busy" + 0.000*"probably buy" + 0.000*"probably call" + 0.000*"probably care" + 0.000*"probably careful" + 0.000*"probably catch" + 0.000*"probably cause" + 0.000*"probably bump"
INFO - 12:59:49: topic #14 (0.020): 0.000*"probably chance" + 0.000*"probably close" + 0.000*"probably busy" + 0.000*"probably buy" + 0.000*"probably call" + 0.000*"probably care" + 0.000*"probably careful" + 0.000*"probably catch" + 0.000*"probably cause" + 0.000*"probably bump"
INFO - 12:59:49: topic #35 (0.020): 0.000*"probably chance" + 0.000*"probably close" + 0.000*"probably busy" + 0.000*"probably buy" + 0.000*"probably call" + 0.000*"probably care" + 0.000*"probably careful" + 0.000*"probably catch" + 0.000*"probably cause" + 0.000*"probably bump"
INFO - 12:59:49: topic diff=0.000000, rho=0.141421
INFO - 12:59:50: -58.908 per-word bound, 540954034076629312.0 perplexity estimate based on a held-out corp

[(20,
  '0.000*"probably chance" + 0.000*"probably close" + 0.000*"probably busy" + 0.000*"probably buy" + 0.000*"probably call" + 0.000*"probably care" + 0.000*"probably careful" + 0.000*"probably catch" + 0.000*"probably cause" + 0.000*"probably bump"'),
 (14,
  '0.000*"probably chance" + 0.000*"probably close" + 0.000*"probably busy" + 0.000*"probably buy" + 0.000*"probably call" + 0.000*"probably care" + 0.000*"probably careful" + 0.000*"probably catch" + 0.000*"probably cause" + 0.000*"probably bump"'),
 (30,
  '0.000*"probably chance" + 0.000*"probably close" + 0.000*"probably busy" + 0.000*"probably buy" + 0.000*"probably call" + 0.000*"probably care" + 0.000*"probably careful" + 0.000*"probably catch" + 0.000*"probably cause" + 0.000*"probably bump"'),
 (40,
  '0.000*"probably chance" + 0.000*"probably close" + 0.000*"probably busy" + 0.000*"probably buy" + 0.000*"probably call" + 0.000*"probably care" + 0.000*"probably careful" + 0.000*"probably catch" + 0.000*"probably cause"

In [13]:
ldan.save('all_subs.model')

INFO - 21:59:39: saving LdaState object under all_subs.model.state, separately None
INFO - 21:59:39: storing np array 'sstats' to all_subs.model.state.sstats.npy
INFO - 21:59:40: saved all_subs.model.state
INFO - 21:59:40: saving LdaMulticore object under all_subs.model, separately ['expElogbeta', 'sstats']
INFO - 21:59:40: storing np array 'expElogbeta' to all_subs.model.expElogbeta.npy
INFO - 21:59:40: not storing attribute dispatcher
INFO - 21:59:40: not storing attribute id2word
INFO - 21:59:40: not storing attribute state
INFO - 21:59:40: saved all_subs.model


In [14]:
import pyLDAvis.gensim as gensimvis
import pyLDAvis

In [22]:
from gensim import corpora
d = corpora.Dictionary()
d.id2token = id2wordn
d.token2id = word2id

In [23]:
vis_data = gensimvis.prepare(ldan, corpusn, d)
pyLDAvis.display(vis_data)

INFO - 22:05:50: Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO - 22:05:50: NumExpr defaulting to 8 threads.
